In [1]:
import numpy as np
import pandas as pd
import psycopg2
from psycopg2 import Error
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
try:
    connection = psycopg2.connect(user="lvf7916",
                                  password="xxx",
                                  host="pg.analytics.northwestern.edu",
                                  port="5432",
                                  database="everything2023")

    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [2]:
tx = pd.read_sql("SELECT * FROM group_5.trnsact;", connection);
tx

,SKU,STORE,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT
0,3,202,2005-01-18,P,1,0.0,30.00,30.00
1,3,202,2005-01-29,R,1,0.0,30.00,30.00
2,3,303,2004-08-18,P,1,0.0,12.00,12.00
3,3,709,2005-08-14,P,1,0.0,30.00,30.00
4,3,802,2005-08-09,P,1,440.0,30.00,30.00
...,...,...,...,...,...,...,...,...
120916891,9999997,7507,2005-04-23,P,1,39.0,39.00,39.00
120916892,9999997,7907,2005-05-03,P,1,39.0,39.00,39.00
120916893,9999997,7907,2005-05-04,R,1,39.0,39.00,39.00
120916894,9999997,7907,2005-05-04,P,1,39.0,39.00,39.00


In [5]:
skst = pd.read_sql("SELECT * FROM group_5.skstinfo;", connection);
skst

,SKU,STORE,COST,RETAIL
0,3,102,123.36,440.00
1,3,103,123.36,440.00
2,3,104,123.36,440.00
3,3,202,123.36,440.00
4,3,203,123.36,440.00
...,...,...,...,...
39230141,9999997,2007,15.00,19.50
39230142,9999997,2707,15.00,9.75
39230143,9999997,3307,15.00,19.50
39230144,9999997,7507,15.00,19.50


In [6]:
strinfo = pd.read_sql("SELECT * FROM group_5.strinfo;", connection);
strinfo

,STORE,CITY,STATE,ZIP
0,2,ST. PETERSBURG,FL,33710
1,3,ST. LOUIS,MO,63126
2,4,LITTLE ROCK,AR,72201
3,7,FORT WORTH,TX,76137
4,9,TEMPE,AZ,85281
...,...,...,...,...
448,9808,GILBERT,AZ,85233
449,9812,METAIRIE,LA,70006
450,9900,LITTLE ROCK,AR,72201
451,9906,LITTLE ROCK,AR,72201


In [4]:
if (connection):
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed") 

In [7]:
skst['SKU'] = skst['SKU'].astype(int)
skst['STORE'] = skst['STORE'].astype(int)
skst.info()

<class 'pandas.core.frame.DataFrame'>
Index: 39230146 entries, 0 to 39230145
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   SKU     int64  
 1   STORE   int64  
 2   COST    float64
 3   RETAIL  float64
dtypes: float64(2), int64(2)
memory usage: 1.5 GB


In [8]:
result_df = pd.merge(skst, tx, on=["SKU", "STORE"], how="right")
result = pd.merge(result_df, strinfo, on=["STORE"], how="left")
result = result.fillna(0)
result

,SKU,STORE,COST,RETAIL,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP
0,3,202,123.36,440.0,2005-01-18,P,1,0.0,30.00,30.00,TAMPA,FL,33612
1,3,202,123.36,440.0,2005-01-29,R,1,0.0,30.00,30.00,TAMPA,FL,33612
2,3,303,0.00,0.0,2004-08-18,P,1,0.0,12.00,12.00,ST. ANN,MO,63074
3,3,709,0.00,0.0,2005-08-14,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308
4,3,802,123.36,440.0,2005-08-09,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,2005-04-23,P,1,39.0,39.00,39.00,HOUSTON,TX,77056
120916892,9999997,7907,15.00,19.5,2005-05-03,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916893,9999997,7907,15.00,19.5,2005-05-04,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916894,9999997,7907,15.00,19.5,2005-05-04,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546


In [2]:
df = result
df

,SKU,STORE,COST,RETAIL,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP
0,3,202,123.36,440.0,2005-01-18,P,1,440.0,30.00,30.00,TAMPA,FL,33612
1,3,202,123.36,440.0,2005-01-29,R,1,440.0,30.00,30.00,TAMPA,FL,33612
2,3,303,0.00,0.0,2004-08-18,P,1,0.0,12.00,12.00,ST. ANN,MO,63074
3,3,709,0.00,0.0,2005-08-14,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308
4,3,802,123.36,440.0,2005-08-09,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,2005-04-23,P,1,39.0,39.00,39.00,HOUSTON,TX,77056
120916892,9999997,7907,15.00,19.5,2005-05-03,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916893,9999997,7907,15.00,19.5,2005-05-04,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546
120916894,9999997,7907,15.00,19.5,2005-05-04,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546


# Set features

In [3]:
df['SALEDATE'] = pd.to_datetime(df['SALEDATE'])
df['Year'] = df['SALEDATE'].dt.year
df['YoYTotalSales'] = df.groupby(['Year', 'STORE'])['RETAIL'].transform('sum')
df['YoYPercentageIncrease'] = df.groupby('STORE')['YoYTotalSales'].pct_change() * 100

df['InventoryTurnover'] = df['COST'] / df['QUANTITY']
df['GrossProfit'] = df['RETAIL'] - (df['COST'] * df['QUANTITY'])
df['YoYGrowth'] = df.groupby('SKU')['AMT'].pct_change(12) * 100
df['ReturnPercentage'] = (df['ORGPRICE'] - df['SPRICE']) / df['ORGPRICE'] * 100

In [4]:
df

,SKU,STORE,COST,RETAIL,SALEDATE,STYPE,QUANTITY,ORGPRICE,SPRICE,AMT,CITY,STATE,ZIP,Year,YoYTotalSales,YoYPercentageIncrease,InventoryTurnover,GrossProfit,YoYGrowth,ReturnPercentage
0,3,202,123.36,440.0,2005-01-18,P,1,440.0,30.00,30.00,TAMPA,FL,33612,2005,4515849.47,NaN,123.36,316.64,NaN,93.181818
1,3,202,123.36,440.0,2005-01-29,R,1,440.0,30.00,30.00,TAMPA,FL,33612,2005,4515849.47,0.000000,123.36,316.64,NaN,93.181818
2,3,303,0.00,0.0,2004-08-18,P,1,0.0,12.00,12.00,ST. ANN,MO,63074,2004,2344233.49,NaN,0.00,0.00,NaN,-inf
3,3,709,0.00,0.0,2005-08-14,P,1,0.0,30.00,30.00,GLENDALE,AZ,85308,2005,9866714.52,NaN,0.00,0.00,NaN,-inf
4,3,802,123.36,440.0,2005-08-09,P,1,440.0,30.00,30.00,CLEARWATER,FL,33761,2005,9012871.83,NaN,123.36,316.64,NaN,93.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120916891,9999997,7507,15.00,19.5,2005-04-23,P,1,39.0,39.00,39.00,HOUSTON,TX,77056,2005,15690624.01,0.000000,15.00,4.50,NaN,0.000000
120916892,9999997,7907,15.00,19.5,2005-05-03,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546,2005,13546626.28,35.299263,15.00,4.50,NaN,0.000000
120916893,9999997,7907,15.00,19.5,2005-05-04,R,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546,2005,13546626.28,0.000000,15.00,4.50,NaN,0.000000
120916894,9999997,7907,15.00,19.5,2005-05-04,P,1,39.0,39.00,39.00,FRIENDSWOOD,TX,77546,2005,13546626.28,0.000000,15.00,4.50,NaN,0.000000


In [5]:
df = df[['SKU', 'STORE', 'STYPE', 'AMT', 'ReturnPercentage', 'InventoryTurnover', 'GrossProfit']]

In [6]:
df.to_csv('final_data_update_by_feature.csv')

In [7]:
df

,SKU,STORE,STYPE,AMT,ReturnPercentage,InventoryTurnover,GrossProfit
0,3,202,P,30.00,93.181818,123.36,316.64
1,3,202,R,30.00,93.181818,123.36,316.64
2,3,303,P,12.00,-inf,0.00,0.00
3,3,709,P,30.00,-inf,0.00,0.00
4,3,802,P,30.00,93.181818,123.36,316.64
...,...,...,...,...,...,...,...
120916891,9999997,7507,P,39.00,0.000000,15.00,4.50
120916892,9999997,7907,P,39.00,0.000000,15.00,4.50
120916893,9999997,7907,R,39.00,0.000000,15.00,4.50
120916894,9999997,7907,P,39.00,0.000000,15.00,4.50


In [ ]:
total_sales_returns = df.groupby(['STORE', 'STYPE'])['AMT'].sum().unstack(fill_value=0)
total_sales_returns['ReturnPercentage'] = (total_sales_returns['R'] / (total_sales_returns['P'] + total_sales_returns['R'])) * 100

df = df.merge(total_sales_returns['ReturnPercentage'].reset_index(), on='STORE', how='left')
df

In [ ]:
df2 = df.groupby('STORE')[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y']].mean()

In [ ]:
df2['success'] = ((df2['InventoryTurnover'] > df2['InventoryTurnover'].quantile(0.50)) &
                          (df2['GrossProfit'] > df2['GrossProfit'].quantile(0.50)) &
                          (df2['ReturnPercentage_y'] > df2['ReturnPercentage_y'].quantile(0.05))).astype(int)

In [ ]:
df2

# Data Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the color palette
sns.set_palette("pastel")

# Explore distribution of 'success' variable with different colors
plt.figure(figsize=(8, 6))
ax = sns.countplot(x='success', data=df2, hue='success')

# Add percentage labels
total = float(len(df2))
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width() / 2., height + 0.1, '{:.2%}'.format(height/total), ha="center")

plt.title('Distribution of Success')
plt.show()




# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(df2['success'].value_counts(), labels=['Not Successful', 'Successful'], autopct='%1.1f%%', colors=['#ff9999','#66b3ff'])

# Add title
plt.title('Success Rate Distribution')

plt.show()


In [ ]:
# Visualize the correlation matrix
corr_matrix = df2.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Pairplot for selected features
sns.pairplot(df2[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y', 'success']], hue='success', diag_kind='kde')
plt.show()

# Data Engineering

In [ ]:
# Create interaction features
df2['Interaction1'] = df2['InventoryTurnover'] * df2['GrossProfit']
#df2['Interaction2'] = df2['ReturnPercentage_y'] / df2['InventoryTurnover']
df2.fillna(df2.mean(), inplace=True)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardize numerical features
scaler = StandardScaler()
df2[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y', 'Interaction1']] = scaler.fit_transform(df2[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y', 'Interaction1']])

In [ ]:
df2

In [ ]:
# Visualize the correlation matrix
corr_matrix = df2.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

# Model Evaluation

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Assuming df2 is your DataFrame
X = df2[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y', 'Interaction1']]
y = df2['success']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use SMOTE to oversample the minority class
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Logit Model

In [ ]:
import statsmodels.api as sm

# Fit logistic regression model on the resampled data
model_resampled = sm.Logit(y_train_resampled, sm.add_constant(X_train_resampled))
result_resampled = model_resampled.fit()

# Display the summary of the logistic regression model
print(result_resampled.summary())

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# Add a constant to the test set
X_test = sm.add_constant(X_test)

# Make predictions on the test set
y_pred_resampled = result_resampled.predict(X_test)
y_pred_binary_resampled = (y_pred_resampled > 0.5).astype(int)

# Evaluate the model on the resampled test set
print("\nAccuracy Score (Resampled):")
print(accuracy_score(y_test, y_pred_binary_resampled))

print("\nConfusion Matrix (Resampled):")
print(confusion_matrix(y_test, y_pred_binary_resampled))

print("\nClassification Report (Resampled):")
print(classification_report(y_test, y_pred_binary_resampled))


## Logistic Regression Hyperparameters

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Define the features
X_train_resampled = df2[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y', 'Interaction1']]
X_test = df2[['InventoryTurnover', 'GrossProfit', 'ReturnPercentage_y', 'Interaction1']]

# Define the target variable
y_train_resampled = df2['success']
y_test = df2['success']

# Create a pipeline for logistic regression
logreg_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('logreg', LogisticRegression())  # Logistic Regression model
])

# Define hyperparameters for logistic regression
logreg_params = {'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Perform grid search for logistic regression
logreg_grid = GridSearchCV(logreg_pipeline, logreg_params, cv=5)
logreg_grid.fit(X_train_resampled, y_train_resampled)

# Print the best hyperparameters for logistic regression
print("Best Hyperparameters for Logistic Regression:", logreg_grid.best_params_)

# Make predictions on the test set using the best logistic regression model
y_pred_logreg = logreg_grid.predict(X_test)

# Evaluate the logistic regression model
print("\nAccuracy Score (Logistic Regression):")
print(accuracy_score(y_test, y_pred_logreg))

print("\nConfusion Matrix (Logistic Regression):")
print(confusion_matrix(y_test, y_pred_logreg))

print("\nClassification Report (Logistic Regression):")
print(classification_report(y_test, y_pred_logreg))

## Random Forest

In [ ]:
# Create a pipeline for random forest classifier
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Standardize features
    ('rf', RandomForestClassifier())  # Random Forest model
])

# Define hyperparameters for random forest
rf_params = {'rf__n_estimators': [50, 100, 200],
             'rf__max_depth': [None, 10, 20, 30]}

# Perform grid search for random forest
rf_grid = GridSearchCV(rf_pipeline, rf_params, cv=5)
rf_grid.fit(X_train_resampled, y_train_resampled)

# Print the best hyperparameters for random forest
print("\nBest Hyperparameters for Random Forest:", rf_grid.best_params_)

# Make predictions on the test set using the best random forest model
y_pred_rf = rf_grid.predict(X_test)

# Evaluate the random forest model
print("\nAccuracy Score (Random Forest):")
print(accuracy_score(y_test, y_pred_rf))

print("\nConfusion Matrix (Random Forest):")
print(confusion_matrix(y_test, y_pred_rf))

print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf))
